In [1]:
import torch
from torch import nn, optim
import numpy as np 
import csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# Preparación de datos

In [2]:
#Obtención de datos crudos
wine_data_path = '../TestData/winequality-white.csv'
wine_data_raw = np.loadtxt(wine_data_path, dtype=np.float32,
delimiter=';', skiprows=1) #Brincar primera linea con nombres de columna

#Obtención de etiquetas de columna
col_list = next(csv.reader(open(wine_data_path), delimiter=';'))

wine_data_raw.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [3]:
#Dividir los datos en tensores
wine_data = torch.from_numpy(wine_data_raw)

data = wine_data[:, :-1]
target = wine_data[:, -1].long()

In [4]:
#Revisión de datos
#e.g. revisar vinos en diferentes rangos de puntaje
bad_indexes = target <= 3 #Produce un tensor de bools para los vinos que cumplen condición
mid_indexes = (target > 3) & (target < 7)
top_indexes = target >= 7
bad_indexes.sum() #Solo 20 vinos con esa calificación

tensor(20)

In [5]:
#Advanced indexing: Corresponde a entradas donde el tensor índice es verdadero
top_data = data[target >= 7] 
mid_data = data[(target > 3) & (target < 7)]
bad_data = data[bad_indexes] 


# Definición de arquitectura

In [6]:
""" class WiNet(nn.Module):
    def __init__(self):
        super(WiNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 30),
            nn.ReLU(),
            nn.Linear(30, 1)
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)
 """
model = nn.Sequential(
    nn.Linear(11, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)
        
model


Sequential(
  (0): Linear(in_features=11, out_features=30, bias=True)
  (1): ReLU()
  (2): Linear(in_features=30, out_features=1, bias=True)
)

# Optimización de la red

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

optimizer = optim.Adam(model.parameters(),
    lr = learning_rate)
loss_fn = nn.MSELoss()

def training_loop(model, epochs, optimizer, loss_fn, train_data, test_data):
    for t in range(epochs):

        loss = loss_fn()
        

        if t % 100 == 0:
            print(t, loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()